In [ ]:
from collections import Counter, deque
from enum import Enum
from itertools import chain
from random import shuffle
from typing import NamedTuple

In [ ]:
def shuffled(l):
    l2 = list(l)
    shuffle(l2)
    return l2


def argmax(l):
    return max(range(len(l)), key=l.__getitem__)


def flatten(l):
    return list(chain.from_iterable(l))


def rotate(l, k):
    d = deque(l)
    d.rotate(k)  # k > 0: right, k < 0: left
    return list(d)


def draw(deck, card_count=2):
    return [deck.pop() for _ in range(card_count)]

In [ ]:
class Auction(NamedTuple):
    offer: list
    bids: list[list]

    @property
    def solution(self):
        bids_value = [sum(bid) for bid in self.bids]
        highest_bid_idx = argmax(bids_value)

        solution = list(self.bids)
        solution[highest_bid_idx] = self.offer
        solution = [self.bids[highest_bid_idx]] + solution
        solution = flatten(solution)

        return solution

In [ ]:
def original_mode_simulation(player_count=4):
    cards_per_player_count = 2 * player_count

    deck = [list(range(cards_per_player_count)) for _ in range(player_count)]

    hands = [shuffled(hand) for hand in deck]

    piles = []
    # Place bids
    for hand in hands:
        piles.append(Auction(offer=draw(hand), bids=[]))

    # Place offers
    for player_idx, hand in enumerate(hands):
        for i in range(1, player_count):
            pile_idx = (i + player_idx) % player_count
            pile = piles[pile_idx]
            pile.bids.append(draw(hand))

    # Resolve auctions
    solutions = []
    for player_idx, pile in enumerate(piles):
        solutions.append(rotate(pile.solution, player_idx))

    # Build hands
    loots = []
    for player_idx in range(player_count):
        loot = [solution[player_idx * 2 : player_idx * 2 + 2] for solution in solutions]
        loot = flatten(loot)
        loots.append(loot)

    return loots

In [ ]:
def recover_mode_simulation(player_count=4):
    cards_per_player_count = 2 * player_count

    deck = [list(range(cards_per_player_count)) for _ in range(player_count)]

    hands = [shuffled(hand) for hand in deck]

    piles = []
    # Place bids
    for hand in hands:
        piles.append(Auction(offer=draw(hand), bids=[]))

    # Run each round
    for pile_idx, pile in enumerate(piles):
        for bidder_idx, hand in enumerate(hands):
            if bidder_idx == pile_idx:
                continue

            pile.bids.append(draw(hand))

        solution = rotate(pile.solution, pile_idx)
        for hand in hands:
            hand += [solution.pop(), solution.pop()]

        hands = [shuffled(hand) for hand in hands]

    return hands

In [ ]:
class GameMode(Enum):
    ORIGINAL = "original"
    RECOVER = "recover"


def simulation(player_count=4, mode=GameMode.ORIGINAL):
    simulator = {
        GameMode.ORIGINAL: original_mode_simulation,
        GameMode.RECOVER: recover_mode_simulation,
    }[mode]

    return simulator(player_count=player_count)

In [ ]:
def game_stats(loots):
    return Counter(flatten([Counter(loot).values() for loot in loots]))


def player_stats(loots):
    stats = Counter()
    for loot in loots:
        # sets is a dictionary-like where (k,v): (set_size, number of sets of that size)
        # e.g. {2:3, 3:1} -> the loot contains two pairs and a three-of-a-kind
        sets = Counter(Counter(loot).values())

        # extract full-houses (a three-of-a-kind and a pair)
        while sets[3] > 0 and sets[2] > 0:
            sets[3] -= 1
            sets[2] -= 1
            sets["full"] += 1

        stats += sets

    return stats

In [ ]:
def average_stats(player_count=4, simulation_count=1_000, mode=GameMode.ORIGINAL):
    count = Counter()

    for _ in range(simulation_count):
        loots = simulation(player_count, mode=mode)
        count += player_stats(loots)

    normalised_count = Counter()

    for k, _ in count.items():
        normalised_count[k] = count[k] / simulation_count

    return normalised_count

In [ ]:
original_stats = average_stats(mode=GameMode.ORIGINAL, simulation_count=1_000_000)
original_stats

In [ ]:
recover_stats = average_stats(mode=GameMode.RECOVER, simulation_count=1_000_000)
recover_stats

In [ ]:
def generate_scoring(stats):
    stats = Counter(stats)
    stats.pop(1)
    stats[2] += stats["full"]
    stats[3] += stats["full"]

    highest_count = stats.most_common()[0][1]

    scoring = {}
    for set, count in stats.most_common():
        if set == 1:
            continue
        scoring[set] = int(highest_count / count)

    return scoring

In [ ]:
generate_scoring(original_stats)

In [ ]:
generate_scoring(recover_stats)